# Pittsburgh Local Stock Health - Data Collection, Formatting, and Feature Engineering

In this notebook, we collect data on local stock prices and and other variables used for this project. We also ensure the data is in the desired format and do some feature engineering.

In [130]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web

In [131]:
# Downloading the stock data from yfinance, each is stored as a dataframe in the list stocks
# Instead of raw values, calculate the monthly percent change so the time series is stationary

tickers = ["EQT", "AA", "PPG", "BK", "PNC", "DKS"]

stocks = {}

for ticker in tickers:
    df = yf.download(ticker, start="2015-01-01", end="2025-07-31", interval="1mo")
    df['Return']=df['Close'].pct_change()
    stocks[ticker]=df[['Close', 'Return']]
    
df.head

C:\Users\benns\AppData\Local\Temp\ipykernel_11940\1288339396.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2025-07-31", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\1288339396.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2025-07-31", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\1288339396.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2025-07-31", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\1288339396.py:9: FutureWarning: YF.download() has changed argumen

<bound method NDFrame.head of Price            Close        High         Low        Open    Volume    Return
Ticker             DKS         DKS         DKS         DKS       DKS          
Date                                                                          
2015-01-01   39.258881   42.489282   36.963395   37.951517  38249500       NaN
2015-02-01   41.113514   42.139640   39.258884   39.380499  18831900  0.047241
2015-03-01   43.317802   44.830389   41.204737   41.273146  26640700  0.053615
2015-04-01   41.343853   45.968942   41.084787   43.271609  22873300 -0.045569
2015-05-01   40.932407   43.385912   39.667557   41.496255  22220900 -0.009952
...                ...         ...         ...         ...       ...       ...
2025-03-01  199.077942  225.992379  183.798442  224.046637  37797100 -0.104576
2025-04-01  186.504974  211.221293  165.275545  199.657866  31887600 -0.063156
2025-05-01  178.160217  223.162217  165.931200  188.422264  47562700 -0.044743
2025-06-01  196.508713

In [132]:
# Combine all the data into one frame
stocks_df=pd.concat({k: v['Return'] for k, v in stocks.items()}, axis =1)
stocks_df.head

<bound method NDFrame.head of                  EQT        AA       PPG        BK       PNC       DKS
Date                                                                  
2015-01-01       NaN       NaN       NaN       NaN       NaN       NaN
2015-02-01  0.072139 -0.054952  0.056084  0.092318  0.093863  0.047241
2015-03-01  0.038722 -0.124856 -0.039075  0.028104  0.013919  0.053615
2015-04-01  0.085314  0.038700 -0.017647  0.052187 -0.016195 -0.045569
2015-05-01 -0.054147 -0.068554  0.033083  0.028169  0.048893 -0.009952
...              ...       ...       ...       ...       ...       ...
2025-03-01  0.112487 -0.082707 -0.028578 -0.057111 -0.084150 -0.104576
2025-04-01 -0.074677 -0.193162 -0.004481 -0.041254 -0.085794 -0.063156
2025-05-01  0.115089  0.091317  0.017821  0.108804  0.092901 -0.044743
2025-06-01  0.061000  0.106114  0.033043  0.028214  0.072550  0.102989
2025-07-01 -0.078361  0.015588 -0.072528  0.113489  0.020652  0.076339

[127 rows x 6 columns]>

In [133]:
#Rename the columns
stocks_df = stocks_df.rename(columns={
    "EQT": "EQT_Monthly_Avg_Return",
    "AA": "AA_Monthly_Avg_Return",
    "PPG": "PPG_Monthly_Avg_Return",
    "BK": "BK_Monthly_Avg_Return",
    "PNC": "PNC_Monthly_Avg_Return",
    "DKS": "DKS_Monthly_Avg_Return",
})
stocks_df.head()

,EQT_Monthly_Avg_Return,AA_Monthly_Avg_Return,PPG_Monthly_Avg_Return,BK_Monthly_Avg_Return,PNC_Monthly_Avg_Return,DKS_Monthly_Avg_Return
Date,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-01,0.072139,-0.054952,0.056084,0.092318,0.093863,0.047241
2015-03-01,0.038722,-0.124856,-0.039075,0.028104,0.013919,0.053615
2015-04-01,0.085314,0.038700,-0.017647,0.052187,-0.016195,-0.045569
2015-05-01,-0.054147,-0.068554,0.033083,0.028169,0.048893,-0.009952


In [134]:
# Downloading Commodity and Local Variable

start_d = "2015-01-01"
end_d = "2025-07-31"

natgas = web.DataReader("MHHNGSP", 'fred', start = start_d, end = end_d)
natgas = natgas.resample("Me").mean()

aluminum = web.DataReader("PALUMUSDM", 'fred', start =start_d, end = end_d)
aluminum = aluminum.resample("M").mean()

oil = web.DataReader("DCOILBRENTEU", "fred", start=start_d, end = end_d)
oil = oil.resample("M").mean()

steel = web.DataReader("WPU1017", 'fred', start = start_d, end = end_d)
steel = steel.resample ("M").mean()

unemployment = web.DataReader("PITT342URN", 'fred', start = start_d, end = end_d)
unemployment = unemployment.resample("M").mean()

avg_hourly = web.DataReader("SMU42383000500000003", 'fred', start = start_d, end = end_d)
avg_hourly = avg_hourly.resample("M").mean()



C:\Users\benns\AppData\Local\Temp\ipykernel_11940\35021271.py:7: FutureWarning: 'Me' is deprecated and will be removed in a future version, please use 'ME' instead.
  natgas = natgas.resample("Me").mean()
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\35021271.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aluminum = aluminum.resample("M").mean()
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\35021271.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  oil = oil.resample("M").mean()
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\35021271.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  steel = steel.resample ("M").mean()
C:\Users\benns\AppData\Local\Temp\ipykernel_11940\35021271.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  unemployment = unem

In [135]:
# Reading Monthly Median home prices from zillow data

zillow = pd.read_csv("C:/Users/benns/Downloads/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_month.csv")
pitt_row = zillow[zillow["RegionName"] == "Pittsburgh, PA"]

pitt_row = pitt_row.drop(columns=["RegionName","RegionID", "SizeRank", "RegionType", "StateName"])
# Data is store as rows with dates as columns, so must reformat
pitt_col = pitt_row.T

pitt_col.columns = ["Median_Home_Value"]
# ensuring all data indexing is by datetime 
pitt_col.index = pd.to_datetime(pitt_col.index)
# Selection of data to align with above
pitt_col = pitt_col[pitt_col.index >= "2015-01-31"]

pitt_col.head

<bound method NDFrame.head of             Median_Home_Value
2015-01-31      136665.337995
2015-02-28      135931.021415
2015-03-31      137251.390593
2015-04-30      138926.683496
2015-05-31      141023.156108
...                       ...
2025-03-31      221789.471969
2025-04-30      224604.746764
2025-05-31      227694.025807
2025-06-30      230342.876943
2025-07-31      232771.348021

[127 rows x 1 columns]>

All of the data is indexed by datetime, so now we can concatenate each of the variables into a dataframe. We also reindex the stock data so that the index is ordered by end of month dates.

In [136]:
stocks_df.index = stocks_df.index - pd.offsets.MonthEnd(0)
stocks_df.head

raw_data = pd.concat([stocks_df, natgas, aluminum, oil, steel, unemployment, avg_hourly, pitt_col], axis = 1)
raw_data= raw_data.rename(columns = {
    "MHHNGSP" : 'natgas',
    'PALUMUSDM' : 'aluminum',
    'DCOILBRENTEU' : 'oil',
    "WPU1017" : 'steel',
    "PITT342URN" : 'unemployment',
    "SMU42383000500000003" : 'avg_hourly',
})
raw_data.head


<bound method NDFrame.head of             EQT_Monthly_Avg_Return  AA_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.072139              -0.054952   
2015-03-31                0.038722              -0.124856   
2015-04-30                0.085314               0.038700   
2015-05-31               -0.054147              -0.068554   
...                            ...                    ...   
2025-03-31                0.112487              -0.082707   
2025-04-30               -0.074677              -0.193162   
2025-05-31                0.115089               0.091317   
2025-06-30                0.061000               0.106114   
2025-07-31               -0.078361               0.015588   

            PPG_Monthly_Avg_Return  BK_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.056084               0.092318   
2015-03-31               -0.039075               0.028

How that the dataframe is assembled, we will want to preprocess to make it suitable for modeling. 

Our first step is to adjust commodity prices and local economic data by inflation rates to capture the trends of these variable accounting for inflation.

Next, we will change each commodity variable to a percent change. This has the effect of standardizing the scale of the data which generally improves model performance for regression data and also we hope to induce some stationary effects on the data. 



In [137]:
# Adjusting non-stock values for inflation

cpi = web.DataReader("CPIAUCNS", 'fred', start =start_d, end = end_d)
cpi.index = cpi.index - pd.offsets.MonthEnd(0)
base_cpi = 233.707
 

raw_data['natgas'] = raw_data['natgas']/cpi['CPIAUCNS'] * base_cpi
raw_data['aluminum'] = raw_data['aluminum']/cpi['CPIAUCNS'] * base_cpi
raw_data['oil'] = raw_data['oil']/cpi['CPIAUCNS'] * base_cpi
raw_data['steel'] = raw_data['steel']/cpi['CPIAUCNS'] * base_cpi
raw_data['avg_hourly'] = raw_data['avg_hourly']/cpi['CPIAUCNS'] * base_cpi
raw_data['Median_Home_Value'] = raw_data['Median_Home_Value']/cpi['CPIAUCNS'] * base_cpi
raw_data.head

<bound method NDFrame.head of             EQT_Monthly_Avg_Return  AA_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.072139              -0.054952   
2015-03-31                0.038722              -0.124856   
2015-04-30                0.085314               0.038700   
2015-05-31               -0.054147              -0.068554   
...                            ...                    ...   
2025-03-31                0.112487              -0.082707   
2025-04-30               -0.074677              -0.193162   
2025-05-31                0.115089               0.091317   
2025-06-30                0.061000               0.106114   
2025-07-31               -0.078361               0.015588   

            PPG_Monthly_Avg_Return  BK_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.056084               0.092318   
2015-03-31               -0.039075               0.028

In [138]:
# Measure percent change in natgas, aluminum, oil, steel, avg_hourly and median home value
var = ['natgas', 'aluminum', 'oil', 'steel', 'avg_hourly', 'Median_Home_Value']

for v in var:
    raw_data[v]=raw_data[v].pct_change()

raw_data.head

C:\Users\benns\AppData\Local\Temp\ipykernel_11940\1777688046.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  raw_data[v]=raw_data[v].pct_change()


<bound method NDFrame.head of             EQT_Monthly_Avg_Return  AA_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.072139              -0.054952   
2015-03-31                0.038722              -0.124856   
2015-04-30                0.085314               0.038700   
2015-05-31               -0.054147              -0.068554   
...                            ...                    ...   
2025-03-31                0.112487              -0.082707   
2025-04-30               -0.074677              -0.193162   
2025-05-31                0.115089               0.091317   
2025-06-30                0.061000               0.106114   
2025-07-31               -0.078361               0.015588   

            PPG_Monthly_Avg_Return  BK_Monthly_Avg_Return  \
2015-01-31                     NaN                    NaN   
2015-02-28                0.056084               0.092318   
2015-03-31               -0.039075               0.028

Finally, we will do some feature engineering for the data. Common features for time series include lagged variables, rolling averages, and seasonality trends. The following function does exactly this.

In [139]:
def features(df):
    for col in df.select_dtypes(include = 'number').columns:

    # generating lag variables
        df[f"{col}_lag1"] = df[col].shift(1)
        df[f"{col}_lag2"] = df[col].shift(2)
    # generates a 3 month rolling average
        df[f"{col}_3mo"] = df[col].rolling(window = 3).mean()
    # generated quarter variable
    df["quarter"]=df.index.quarter

    return df

In [140]:
raw_data = features(raw_data)
raw_data.head()

,EQT_Monthly_Avg_Return,AA_Monthly_Avg_Return,PPG_Monthly_Avg_Return,BK_Monthly_Avg_Return,PNC_Monthly_Avg_Return,DKS_Monthly_Avg_Return,natgas,aluminum,oil,steel,...,unemployment_lag1,unemployment_lag2,unemployment_3mo,avg_hourly_lag1,avg_hourly_lag2,avg_hourly_3mo,Median_Home_Value_lag1,Median_Home_Value_lag2,Median_Home_Value_3mo,quarter
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2015-02-28,0.072139,-0.054952,0.056084,0.092318,0.093863,0.047241,-0.044284,-0.002623,0.211158,-0.031207,...,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2015-03-31,0.038722,-0.124856,-0.039075,0.028104,0.013919,0.053615,-0.019771,-0.029954,-0.043733,-0.025174,...,6.0,6.2,6.000000,0.008242,NaN,NaN,-0.009674,NaN,NaN,1
2015-04-30,0.085314,0.038700,-0.017647,0.052187,-0.016195,-0.045569,-0.079610,0.023470,0.062951,-0.032932,...,5.8,6.0,5.566667,-0.001787,0.008242,0.000650,0.003740,-0.009674,0.001406,2
2015-05-31,-0.054147,-0.068554,0.033083,0.028169,0.048893,-0.009952,0.086416,-0.013356,0.070992,-0.025902,...,4.9,5.8,5.366667,-0.004506,-0.001787,-0.004613,0.010153,0.003740,0.007945,2


In [141]:
raw_data[raw_data.isna().any(axis =1)]

,EQT_Monthly_Avg_Return,AA_Monthly_Avg_Return,PPG_Monthly_Avg_Return,BK_Monthly_Avg_Return,PNC_Monthly_Avg_Return,DKS_Monthly_Avg_Return,natgas,aluminum,oil,steel,...,unemployment_lag1,unemployment_lag2,unemployment_3mo,avg_hourly_lag1,avg_hourly_lag2,avg_hourly_3mo,Median_Home_Value_lag1,Median_Home_Value_lag2,Median_Home_Value_3mo,quarter
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2015-02-28,0.072139,-0.054952,0.056084,0.092318,0.093863,0.047241,-0.044284,-0.002623,0.211158,-0.031207,...,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2015-03-31,0.038722,-0.124856,-0.039075,0.028104,0.013919,0.053615,-0.019771,-0.029954,-0.043733,-0.025174,...,6.0,6.2,6.0,0.008242,NaN,NaN,-0.009674,NaN,NaN,1
2025-07-31,-0.078361,0.015588,-0.072528,0.113489,0.020652,0.076339,0.058005,0.000000,-0.007146,-0.006323,...,4.2,3.6,NaN,0.006401,-0.004925,0.005321,0.008196,0.011641,0.009619,3


Now that we've completed feature engineering, our last step is to remove rows with any null values. This may exclude some of the rows at the beginning of the model and the end, but most of the data points will be retained. Then we save the file to a CSV.

In [ ]:
raw_data = raw_data.dropna()

,EQT_Monthly_Avg_Return,AA_Monthly_Avg_Return,PPG_Monthly_Avg_Return,BK_Monthly_Avg_Return,PNC_Monthly_Avg_Return,DKS_Monthly_Avg_Return,natgas,aluminum,oil,steel,...,unemployment_lag1,unemployment_lag2,unemployment_3mo,avg_hourly_lag1,avg_hourly_lag2,avg_hourly_3mo,Median_Home_Value_lag1,Median_Home_Value_lag2,Median_Home_Value_3mo,quarter
